<style>
p {
    line-height: 1.5em;
    font-size: 20px;
}
</style>
---
<center>
<h1>第十一课</h1>
</center>

## **主题**：案例分析：决策树的应用

## 纲要
### 1. 决策树的工作原理
### 2. 决策树的生成过程
### 3. 构造一个决策分类树和一个决策回归树
---


<center><h2>1. 决策树的工作原理</h2></center>

<p>
在现实生活中，我们会遇到各种选择，不论挑选水果，还是网上购物，都是基于以往的经验来做判断。如果把判断背后的逻辑整理成一个结构图，会发现它实际上是一个树状图，这就涉及到决策树模型。
</p>

<p>
比如我们要出门打篮球，一般会根据“天气”、“温度”、“湿度”、“刮风”这几个条件来判断，最后得到结果：去还是不去。
</p>

<center><img src="../../image/playbasketball.png" /></center>

<p>
建立决策树模型的过程，一般要经历两个阶段：<b>构造</b>和<b>剪枝</b>。
</p>

<p>
<b>构造的阶段——就是选择什么属性作为节点的过程</b>。在构造过程中，会存在三种节点：
</p>

<p>1. 根节点：就是树的最顶端，最开始的那个节点。在上图中，“天气”就是一个根节点；</p>

<p>2. 内部节点：就是树中间的那些节点，比如说“温度”、“湿度”、“刮风”；</p>

<p>3. 叶节点：就是树最底部的节点，也就是决策结果。</p>

<p>
<b>剪枝的阶段</b>——就是剪掉决策树中不太必要的节点。目的是防止“过拟合”（Overfitting）现象的发生。引起过拟合的原因主要是训练集太过理想，而出现分析结果很“呆板”的情况，最终导致泛化能力较弱。泛化能力指的分类器是通过训练集抽象出来的分类能力，可以理解是举一反三的能力。
</p>

<p>
剪枝操作可分为预剪枝和后剪枝两种。其中，预剪枝是在构造决策树时进行，后剪枝是在生成决策树之后进行。
</p>

<p>
对于上面到底要不要去打篮球的问题，显然将哪个属性（天气、温度、湿度、刮风）作为根节点是个关键问题，在这里我们先介绍两个指标：<b>纯度</b>和<b>信息熵</b>。
</p>

<p>
决策树的构造过程理解成为寻找纯净划分的过程。数学上，用纯度来表示，纯度换一种方式来解释就是让目标变量的分歧最小。
</p>

<p>
举个例子，假设有 3 个集合：</p>
<p><b>集合 1</b>：6 次都去打篮球；</p>
<p><b>集合 2</b>：4 次去打篮球，2 次不去打篮球；</p>
<p><b>集合 3</b>：3 次去打篮球，3 次不去打篮球。</p>

<p>按照纯度指标来说，集合 1> 集合 2> 集合 3。因为集合 1 的分歧最小，集合 3 的分歧最大。
</p>

<p>
<B>信息熵</B>（entropy）表示的是信息的不确定度。在信息论中，随机离散事件出现的概率存在着不确定性。为了衡量这种信息的不确定性，信息学之父香农引入了信息熵的概念，并给出了计算信息熵的数学公式。它能帮我们反映出来这个信息的不确定度。当不确定性越大时，它所包含的信息量也就越大，信息熵也就越高。
</P>

<p>构造决策树的时候，一般会基于纯度来构建。而经典的 “不纯度”的指标有三种，分别是信息增益（ID3 算法）、信息增益率（C4.5 算法）以及基尼指数（Cart 算法）。</p>

<center><img src="../../image/decidetree.png" /></center>

<p>
我们重点来看下CART算法。CART 算法，英文全称叫做 Classification And Regression Tree，中文叫做分类回归树。ID3 和 C4.5 算法可以生成二叉树或多叉树，而 CART 只支持二叉树。同时 CART 决策树比较特殊，既可以作分类树，又可以作回归树。
</p>

<p>

在Python中程序包sklearn可用来构建CART分类树和CART回归树。下面针对sklearn自带的葡萄酒数据集构造一棵分类决策树，并对sklearn 自带的波士顿房价数据集构造一棵回归树进行房价预测。
</p>

In [1]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import tree
from sklearn.datasets import load_wine
from IPython.display import SVG
from graphviz import Source
from IPython.display import display                               
from ipywidgets import interactive
# 加载葡萄酒数据集
data = load_wine()
# 获取特征矩阵
X = data.data
# 获取目标向量
y = data.target
# 获取分类标识
labels = data.feature_names
# 打印数据集描述
print(data.DESCR)
def plot_tree(crit, split, depth, min_split, min_leaf=0.2):
    estimator = DecisionTreeClassifier(random_state = 0 
      , criterion = crit
      , splitter = split
      , max_depth = depth
      , min_samples_split=min_split
      , min_samples_leaf=min_leaf)
    estimator.fit(X, y)
    graph = Source(tree.export_graphviz(estimator
        , out_file=None
        , feature_names=labels
        , class_names=['0', '1', '2']
        , filled = True))
    
    display(SVG(graph.pipe(format='svg')))
    return estimator

inter = interactive(plot_tree 
   , crit = ["gini", "entropy"]
   , split = ["best", "random"]
   , depth=[1,2,3,4]
   , min_split=(0.1,1)
   , min_leaf=(0.1,0.5))
display(inter)

.. _wine_dataset:

Wine recognition dataset
------------------------

**Data Set Characteristics:**

    :Number of Instances: 178 (50 in each of three classes)
    :Number of Attributes: 13 numeric, predictive attributes and the class
    :Attribute Information:
 		- Alcohol
 		- Malic acid
 		- Ash
		- Alcalinity of ash  
 		- Magnesium
		- Total phenols
 		- Flavanoids
 		- Nonflavanoid phenols
 		- Proanthocyanins
		- Color intensity
 		- Hue
 		- OD280/OD315 of diluted wines
 		- Proline

    - class:
            - class_0
            - class_1
            - class_2
		
    :Summary Statistics:
    
    ============================= ==== ===== ======= =====
                                   Min   Max   Mean     SD
    ============================= ==== ===== ======= =====
    Alcohol:                      11.0  14.8    13.0   0.8
    Malic Acid:                   0.74  5.80    2.34  1.12
    Ash:                          1.36  3.23    2.36  0.27
    Alcalinity of Ash:            1

interactive(children=(Dropdown(description='crit', options=('gini', 'entropy'), value='gini'), Dropdown(descri…

In [3]:

# encoding=utf-8
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.tree import DecisionTreeRegressor
# 准备数据集
boston=load_boston()
# 探索数据
print(boston.feature_names)
# 获取特征集和房价
features = boston.data
prices = boston.target
# 随机抽取33%的数据作为测试集，其余为训练集
train_features, test_features, train_price, test_price = train_test_split(features, prices, test_size=0.33)
# 创建CART回归树
dtr=DecisionTreeRegressor()
# 拟合构造CART回归树
dtr.fit(train_features, train_price)
# 预测测试集中的房价
predict_price = dtr.predict(test_features)
# 测试集的结果评价
print('回归树二乘偏差均值:', mean_squared_error(test_price, predict_price))
print('回归树绝对值偏差均值:', mean_absolute_error(test_price, predict_price)) 

['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
回归树二乘偏差均值: 17.01802395209581
回归树绝对值偏差均值: 3.0652694610778437


<center><img src="../../image/CART.png" ></cetner>